In [6]:
import iesta.processor as proc  
import iesta.properties as properties  
import pandas as pd

In [7]:
liberal_arguments_df, liberal_not_found = processor.get_ideology_based_voter_participant_df(properties.LIBERAL_IDEOLOGY)
#conservative_arguments_df, conservative_not_found = processor.get_ideology_based_voter_participant_df(prop.CONSERVATIVE_IDEOLOGY)
liberal_arguments_df.head()

,id,debate_id,p_name,effects,effect_count,top_effect,effect,category,round,argument
0,.-.-.-Ha-YOURE-GOING-DOWN-BEEM0R/1/|Logical-Ma...,.-.-.-Ha-YOURE-GOING-DOWN-BEEM0R/1/,Logical-Master,ineffective,"b'{""ineffective"": 1}'",ineffective,ineffective,Miscellaneous,0,\n \r\nRESOLVED: It is impossible not to cons...
1,.-.-.-Ha-YOURE-GOING-DOWN-BEEM0R/1/|Logical-Ma...,.-.-.-Ha-YOURE-GOING-DOWN-BEEM0R/1/,Logical-Master,ineffective,"b'{""ineffective"": 1}'",ineffective,ineffective,Miscellaneous,1,"\n \r\nOkay, just now reading the round, it w..."
2,.-.-.-Ha-YOURE-GOING-DOWN-BEEM0R/1/|Logical-Ma...,.-.-.-Ha-YOURE-GOING-DOWN-BEEM0R/1/,Logical-Master,ineffective,"b'{""ineffective"": 1}'",ineffective,ineffective,Miscellaneous,2,\n \r\n--->RE:Since my opponent has left it u...
3,.-.-.-Ha-YOURE-GOING-DOWN-BEEM0R/1/|Logical-Ma...,.-.-.-Ha-YOURE-GOING-DOWN-BEEM0R/1/,Logical-Master,ineffective,"b'{""ineffective"": 1}'",ineffective,ineffective,Miscellaneous,3,\n \r\nCONSCIOUS SECTION: \n \r\nLadies and...
4,.-.-.-Ha-YOURE-GOING-DOWN-BEEM0R/1/|beem0r,.-.-.-Ha-YOURE-GOING-DOWN-BEEM0R/1/,beem0r,ineffective,"b'{""ineffective"": 1}'",ineffective,ineffective,Miscellaneous,0,\n \r\nI would eplain the vast differences be...


In [31]:
#len(liberal_arguments_df['debate_id'].unique())

debates = liberal_arguments_df['debate_id'].unique()

     

len(validation_debates)


2493

In [32]:
validation_debates, test_debates = train_test_split(testval_debates,test_size=0.3, random_state=2, shuffle=True) 

,debate_id,split
0,Is-Gaming-a-Sport/1/,validation
1,Increased-restrictions-should-be-put-on-guns-i...,validation
2,Darth-Maul-was-the-best-Sith/1/,validation
3,LM-Classic-mongeese-vs-iamadragon/1/,validation
4,Che-Guevara/1/,validation
...,...,...
2488,ETHICS-of-Polygamy/1/,validation
2489,Literal-Creationism-Pro-VS.-Theistic-Evolution...,validation
2490,Capitalism-is-one-of-the-main-reasons-for-suff...,validation
2491,animal-habitat-loss-because-of-industry/1/,validation


In [116]:
from sklearn.model_selection import train_test_split
import numpy
from tqdm import tqdm
import os
import iesta.utils as utils
def split_iesta_dataset_by_debate(ideology:str,  test_split=0.3, abstract_effect:bool=False, random_state:int=2 ):
    def _abstract_effect(row):
        if row['effect'] != "effective":
            row['effect'] = "ineffective"
        return row
    
    abstract_st = "_abstracted" if abstract_effect else ""
    file_path = os.path.join(properties.ROOT_PATH,  f"splitted_{ideology}_debate_arguments_w{abstract_st}_effect_test{test_split}_random{random_state}.parquet")
    tqdm.pandas()

    if os.path.isfile(file_path):
        data_w_splits_df = pd.read_parquet(file_path)
        split_effect_pivot_df = pd.crosstab(data_w_splits_df['split'], data_w_splits_df['effect'])
        return data_w_splits_df, split_effect_pivot_df
        
    processor = proc.Process()
    df, _ = processor.get_ideology_based_voter_participant_df(ideology)
    debates = df['debate_id'].unique()

    training_debate, testval_debates = train_test_split(debates,test_size=test_split, random_state=random_state, shuffle=True) 
    validation_debates, test_debates = train_test_split(testval_debates,test_size=(1/3), random_state=random_state, shuffle=True) 
    
    splits_df_lst = []
    splits_df_lst.append(pd.DataFrame({'debate_id': training_debate, 'split': ['training']*len(training_debate)}))
    splits_df_lst.append(pd.DataFrame({'debate_id': validation_debates, 'split': ['validation']*len(validation_debates)}))
    splits_df_lst.append(pd.DataFrame({'debate_id': test_debates, 'split': ['test']*len(test_debates)}))
    
    debate_split_df  = pd.concat(splits_df_lst)
    
    data_w_splits_df = df.merge(right=debate_split_df, how='left', on='debate_id', suffixes=('', 'extra') )
    
    # Validation
    assert len(debate_split_df) == len(debates)
    assert len(data_w_splits_df['debate_id'].unique()) == len(debates)
    for split in ["training", "validation", "test"]:
        assert len(data_w_splits_df[data_w_splits_df['split'] == split]['debate_id'].unique()) == \
        len(debate_split_df[debate_split_df['split'] == split])

    # Abstract
    if abstract_effect:
        data_w_splits_df = data_w_splits_df.apply(_abstract_effect, axis=1)
    
    split_effect_pivot_df = pd.crosstab(data_w_splits_df['split'], data_w_splits_df['effect'])
    print(split_effect_pivot_df)
    data_w_splits_df.to_parquet(file_path)
    return data_w_splits_df, split_effect_pivot_df
    
    

In [168]:
def clean_argument(arg):
    arg = arg.strip()
    #replace smileys
    # replace URLS
    return arg
def apply_add_rounds(row, df__):
    row['rounds'] = df__["round"].tolist()
    return row

def pepare_data_for_transformers(ideology:str,
                                 test_split=0.3,
                                 methodology:str="each", # first, last, firstlast, currentprevious
                                 abstract_effect:bool=False ):
    data_w_splits_df, split_effect_pivot_df = split_iesta_dataset_by_debate(ideology, test_split, abstract_effect)
    
    result_df_lst = []
    abstract_st = "abstracted" if abstract_effect else ""
    path = os.path.join(properties.ROOT_PATH, "splitted", abstract_st, f"methodology_{methodology}")
    
    tqdm.pandas()
    for effect_split, effect_split_df in data_w_splits_df.groupby(["effect", "split"]):
        # for an effect
        args_lst = []
        temp_df = pd.DataFrame()
        
        utils.create_folder(path)
        file = os.path.join(path, f"{ideology}_{effect_split[0]}_{effect_split[1]}.txt")        
        
        if methodology == "each":
            args_lst = effect_split_df["argument"].tolist()
            df_ =effect_split_df.copy() 
            result_df_lst.append(df_)

        elif methodology in ["first", "last"]:
            for group, df_ in effect_split_df.groupby(["debate_id", "p_name"]):
                debate_round = df_["round"].min() if methodology == "first" else df_["round"].max()
                
                df_ = df_[df_["round"] == debate_round]
                assert len(df_["argument"].tolist()) == 1
                result_df_lst.append(df_)
                args_lst.extend(df_["argument"].tolist())
                
        elif methodology in ["firstlast"]:
            for group, df_ in effect_split_df.groupby(["debate_id", "p_name"]):
                
                df_ = df_[(df_["round"] == df_["round"].min()) | (df_["round"] == df_["round"].max())]
                df_ = df_.sort_values(by="round")
                
                arg = " <LAST> ".join(df_["argument"].tolist())
                assert len(df_["argument"].tolist()) == 2 if ( df_["round"].min() !=  df_["round"].max()) else \
                len(df_["argument"].tolist()) == 1 
                
                temp_df = df_[-1:][['id',
                                     'debate_id',
                                     'p_name',
                                     'effect',
                                     'category',
                                     'round',
                                     'argument',
                                     'split']].copy()
                
                
                temp_df = temp_df.apply(apply_add_rounds,  args=(df_,), axis=1,)
                temp_df['argument'] = arg
                result_df_lst.append(temp_df)
                args_lst.append(arg) 
                
        elif methodology in ["currentprevious"]:
            #for group, df_ in data_w_splits_df.groupby(["debate_id", "round"]):
            df_ = effect_split_df.apply(_apply_add_prev_arg,  args=(data_w_splits_df,), axis=1)
            args_lst.extend(df_["current_arg_w_previous"].tolist())
            result_df_lst.append(df_)
            
            
        with open(file, 'w', encoding="utf8") as f:
            for arg in args_lst:
                arg = f"<s>{clean_argument(arg)}</s>"
                f.write(f"{arg}\n")
        
    df = pd.concat(result_df_lst)
    df_file = os.path.join(path, f"processed_data_{ideology}.parquet")
    df.to_parquet(df_file)
    return df, pd.crosstab(df['split'], df['effect'])
            


def get_previous_opposite_arg(df, debate_id, p_name, debate_round ):
    prev_arg = ""
    if debate_round > 0:
        prev_arg_df = df[(df["debate_id"] == debate_id) & ~(df["p_name"] == p_name) & (df["round"] == (debate_round-1))]
        if len(prev_arg_df) == 0:
            with open("../data/error_no_previous_args.txt", "a") as f:
                f.write(f"{debate_id},  p_name:{p_name}, round:{debate_round}\n")
        
        prev_arg =  prev_arg_df["argument"].values[0] if len(prev_arg_df)>0 else ""
    return prev_arg

def _apply_add_prev_arg(row, original_df):
    debate_id = row['debate_id']
    p_name = row['p_name']
    debate_round = row['round']

    previous_opposite_arg = get_previous_opposite_arg(original_df, debate_id, p_name, debate_round )    
    row["current_arg_w_previous"] = f"{previous_opposite_arg} <MAIN-ARG> {row['argument']}"
    
    return row

In [173]:
methodologies = ["each", "first", "last", "firstlast", "currentprevious"]
ideologies =  [properties.CONSERVATIVE_IDEOLOGY, properties.LIBERAL_IDEOLOGY]
splitted_data={}

In [174]:



for ideology in ideologies:
    #splitted_data[ideology] = {} if ideology not in splitted_data.keys() else splitted_data[ideology]
    for methodology in methodologies:
        #splitted_data[ideology][methodology] = {} if methodology not in splitted_data[ideology].keys() else splitted_data[ideology][methodology]
        
        desc = f"Ideology: {ideology} - preprocessing method: {methodology}"
        print("\n\n" +"-"*len(desc))
        print(desc)
        print("-"*len(desc))
        _, pivot_df = pepare_data_for_transformers(ideology = ideology,
                                 test_split=0.3,
                                 methodology=methodology, # first, last, firstlast, currentprevious
                                 abstract_effect=False ) 
        
        print(pivot_df)

        _, pivot_df = pepare_data_for_transformers(ideology = ideology,
                                 test_split=0.3,
                                 methodology=methodology, # first, last, firstlast, currentprevious
                                 abstract_effect=True ) 
        print(pivot_df)




---------------------------------------------------
Ideology: Conservative - preprocessing method: each
---------------------------------------------------
effect      effective  ineffective   okay  provocative
split                                                 
test              294         8164   1677           93
training         2044        57196  11460          616
validation        649        16364   3178          197
effect      effective  ineffective
split                             
test              294         9934
training         2044        69272
validation        649        19739


----------------------------------------------------
Ideology: Conservative - preprocessing method: first
----------------------------------------------------
effect      effective  ineffective  okay  provocative
split                                                
test               98         2853   535           31
training          612        20119  3705          210
validation     